**Download necessary resources**

In [0]:
! wget --no-check-certificate https://storage.googleapis.com/laurencemoroney-blog.appspot.com/rps.zip -O /tmp/rps.zip

--2019-10-06 12:14:30--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/rps.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.212.128, 2607:f8b0:4001:c1d::80
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.212.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 200682221 (191M) [application/zip]
Saving to: ‘/tmp/rps.zip’

/tmp/rps.zip        100%[===================>] 191.38M  56.8MB/s    in 3.4s    

2019-10-06 12:14:33 (56.8 MB/s) - ‘/tmp/rps.zip’ saved [200682221/200682221]



In [0]:
! wget --no-check-certificate https://storage.googleapis.com/laurencemoroney-blog.appspot.com/rps-test-set.zip -O /tmp/rps-test-set.zip

--2019-10-06 12:14:36--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/rps-test-set.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.111.128, 2607:f8b0:4001:c05::80
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.111.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 29516758 (28M) [application/zip]
Saving to: ‘/tmp/rps-test-set.zip’

/tmp/rps-test-set.z 100%[===================>]  28.15M  63.4MB/s    in 0.4s    

2019-10-06 12:14:37 (63.4 MB/s) - ‘/tmp/rps-test-set.zip’ saved [29516758/29516758]



**Unzipping downloaded files**

In [0]:
import os
import zipfile

local_zip = '/tmp/rps.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp/')
zip_ref.close()

local_zip = '/tmp/rps-test-set.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp/')
zip_ref.close()

**Generating training set**

In [0]:
from  keras.preprocessing.image import ImageDataGenerator
import keras.preprocessing.image

TRAINING_DIR = '/tmp/rps/'

training_datagen = ImageDataGenerator(rescale = 1./255)
train_generator = training_datagen.flow_from_directory(
                  TRAINING_DIR,
                  target_size = (150, 150),
                  class_mode = 'categorical')

Using TensorFlow backend.


Found 2520 images belonging to 3 classes.


**Generating validation set**

In [0]:
VALIDATION_DIR = '/tmp/rps-test-set/'

validation_datagen = ImageDataGenerator(rescale = 1./255)
validation_generator = validation_datagen.flow_from_directory(
                  VALIDATION_DIR,
                  target_size = (150, 150),
                  class_mode = 'categorical')

Found 372 images belonging to 3 classes.


**Prepare the model**

In [0]:
import tensorflow as tf

model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(150, 150, 3)),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        
        # flatten the results to feed into a DNN

        tf.keras.layers.Flatten(),
        tf.keras.layers.Dropout(0.5),

        # 512 hidden neuron layer

        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(3, activation='softmax')])

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

history = model.fit_generator(train_generator, epochs=25,
                              validation_data = validation_generator,
                              workers=4,
                              verbose=1)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Epoch 1/25
79/79 [==============================] - 176s 2s/step - loss: 0.5532 - acc: 0.7595 - val_loss: 0.5912 - val_acc: 0.8790
Epoch 2/25
79/79 [==============================] - 178s 2s/step - loss: 0.1300 - acc: 0.9766 - val_loss: 0.8357 - val_acc: 0.8253
Epoch 3/25
79/79 [==============================] - 172s 2s/step - loss: 0.0068 - acc: 0.9984 - val_loss: 1.0468 - val_acc: 0.8629
Epoch 4/25
79/79 [==============================] - 171s 2s/step - loss: 0.0300 - acc: 0.9944 - val_loss: 1.3489 - val_acc: 0.7984
Epoch 5/25
79/79 [==============================] - 172s 2s/step - loss: 5.7897e-04 - acc: 1.0000 - val_loss: 2.1182 - val_acc: 0.8333
Epoch 6/25
79/79 [==============================] - 173s 2s/step - loss: 0.0420 - acc: 0.9909 - val_loss: 1.8540 - val_acc: 0.8280
Epoch 7/25
79/79 [==============================] - 174s 2s/step - loss: 2.9597e-04 - acc: 1

In [0]:
model.save('rps.h5')